In [11]:
# Import Required Libraries

import pandas as pd
import datetime as dt
import openpyxl as exl
from dateutil.relativedelta import relativedelta
import math

# Read Disbursal Data from Excel Sheet
sample_disbursal=pd.read_excel('C:/Users/Vinayak.Patil/Downloads/Sample_loan_disbursal.xlsx')

In [12]:
# To view sample data
sample_disbursal.head()

,Loan No.,Loan Tenure (Years),Loan Tenure (months),ROI %,Loan Amount,Disbursement Date
0,LN_001,3,36,0.1200,120000,2024-01-21
1,LN_002,2,24,0.1450,200000,2024-02-10
2,LN_003,1,12,0.1350,450000,2024-02-15
3,LN_004,3,36,0.0935,1100000,2024-02-28
4,LN_005,3,36,0.1500,500000,2024-03-25


In [13]:
# Check Datatype of columns 
sample_disbursal.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15 entries, 0 to 14
Data columns (total 6 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   Loan No.              15 non-null     object        
 1   Loan Tenure (Years)   15 non-null     int64         
 2   Loan Tenure (months)  15 non-null     int64         
 3   ROI %                 15 non-null     float64       
 4   Loan Amount           15 non-null     int64         
 5   Disbursement Date     15 non-null     datetime64[ns]
dtypes: datetime64[ns](1), float64(1), int64(3), object(1)
memory usage: 852.0+ bytes


In [14]:
rps_table=[]
for index, row in sample_disbursal.iterrows():
    loan_amt = float(row['Loan Amount'])
    opening_balance = loan_amt
    loan_id = row['Loan No.']
    ROI_yr = float(row['ROI %'])
    roi_monthly=ROI_yr/12
    Tenure_yr = float(row['Loan Tenure (Years)'])
    
    # Calculate EMI start_date
    emi_start_date= row['Disbursement Date'] +  relativedelta(months=1)
    
    # Calculate EMI
    emi = round((loan_amt*roi_monthly*pow(1+roi_monthly,Tenure_yr*12)/(pow(1+roi_monthly,Tenure_yr*12) -1)),2)
    
    for i in range(1,int(Tenure_yr)*12+1):
        interest=round((opening_balance*roi_monthly),2)
        principal=round((emi-interest),2)
        
        if i<int(Tenure_yr)*12:
            closing_balance=round((opening_balance-principal),2)
            
        elif i==int(Tenure_yr)*12:
            closing_balance = 0
        
        next_due_date=emi_start_date+relativedelta(months=i-1)
        rps_data=[loan_id, loan_amt, ROI_yr,Tenure_yr, next_due_date.strftime("%Y-%m-%d") , i ,interest , principal,round(interest+principal,2), closing_balance]
        opening_balance=closing_balance
        rps_table.append(rps_data)

# Create Dataframe and give name to respective columns
final_rps=pd.DataFrame(rps_table,columns=['loan_id', 'Loan Amount','ROI','Tenure (Years)', 'Due Date','Installment No','Interest','Principal','EMI', 'Closing balance'])
# Download Data in Excel file
final_rps.to_excel('C:/Users/Vinayak.Patil/Downloads/Repayment_schedule_final.xlsx',index=False)

In [15]:
final_rps

,loan_id,Loan Amount,ROI,Tenure (Years),Due Date,Installment No,Interest,Principal,EMI,Closing balance
0,LN_001,120000.0,0.12,3.0,2024-02-21,1,1200.00,2785.72,3985.72,117214.28
1,LN_001,120000.0,0.12,3.0,2024-03-21,2,1172.14,2813.58,3985.72,114400.70
2,LN_001,120000.0,0.12,3.0,2024-04-21,3,1144.01,2841.71,3985.72,111558.99
3,LN_001,120000.0,0.12,3.0,2024-05-21,4,1115.59,2870.13,3985.72,108688.86
4,LN_001,120000.0,0.12,3.0,2024-06-21,5,1086.89,2898.83,3985.72,105790.03
...,...,...,...,...,...,...,...,...,...,...
607,LN_015,450000.0,0.17,5.0,2029-07-19,56,759.59,10424.07,11183.66,43193.97
608,LN_015,450000.0,0.17,5.0,2029-08-19,57,611.91,10571.75,11183.66,32622.22
609,LN_015,450000.0,0.17,5.0,2029-09-19,58,462.15,10721.51,11183.66,21900.71
610,LN_015,450000.0,0.17,5.0,2029-10-19,59,310.26,10873.40,11183.66,11027.31
